In [ ]:
# Authenticate google account

from google.colab import auth
auth.authenticate_user()

In [ ]:
# Install OpenAI
!pip install openai

In [ ]:
import openai
import re
import os
from google.cloud import bigquery

openai_api_key = "your open api key"
openai.api_key = openai_api_key
os.environ["OPENAI_API_KEY"] = openai_api_key

# Handle OpenAI Rate limits
openai_rpm = 3
t2c_sleep = 60/openai_rpm
t2c_sleep = t2c_sleep if t2c_sleep >= 1 else 0

In [ ]:
# Set-up all the modules with engineered prompts.
import time
pretext= 'Make sure to use regex. Please be as specific as possible and only return the final query enclosed in ```. Do not provide explanations. Using the table: bigquery-public-data.idc_current.dicom_all: '
def text2cohort(prompt):

  # Setup: Priming the LLM to understand the task
  messages = []
  content = pretext
  messages.append({"role": "user", "content": content})
  completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0301",
    messages = messages
  )
  chat_response = completion.choices[0].message.content
  messages.append({"role": "assistant", "content": chat_response})

  # Again prime the LLM + get response for your task

  messages.append({"role": "user", "content": prompt})
  completion = openai.ChatCompletion.create(
    model = "gpt-3.5-turbo-0301",
    messages = messages
  )
  chat_response = completion.choices[0].message.content
  messages.append({"role": "assistant", "content": chat_response})
  print(chat_response)

  # Parse the chat response and get the query
  query = parse(chat_response)
  print(query)

  # run the big query and get the response dataframe 
  df, e, success = runBigQuery(query[0])
  k = 1
  time.sleep(t2c_sleep)
  while success == 0 and k < 10:
    messages.append({"role": "user", "content": str(e)})
    completion = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo-0301",
      messages = messages
    )
    chat_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": chat_response})
    print(chat_response)
    query = parse(chat_response)
    print(query)
    df, e, success = runBigQuery(query[0])
    k += 1
    time.sleep(t2c_sleep)
  return df

def parse(chat_response):
  code_blocks = re.findall(r"```(.+?)```", chat_response, re.DOTALL)
  return code_blocks
  

def runBigQuery(query):
  # Construct a BigQuery client object.
  e = 0
  df = 0
  success = 0
  try:
    client = bigquery.Client(project='Replace with your GCP Project ID')
    df = client.query(query).to_dataframe()
    print(df.head())
    success = 1
  except Exception as e:
    print(str(e))
    success = 0
    return df, e, success
  return df, e, success

In [ ]:
# Text2Cohort
df = text2cohort(pretext+'List all collections that contain segmentation images')